In [1]:
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install matplotlib
%pip install dash==2.14.2 jupyter-dash==0.4.2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [ ]:
# get the data ready

In [4]:
pay=pd.read_csv(r"C:\Users\silve\Desktop\Data Science\Projects\Trade Franchises\Meta Data\Raw data\Payments.csv")
pay.head(1)

,Store_ID,Missed_Payments_12mo,Missed_Payments_6mo,Missed_Payments_3mo,Initial_Fee_Paid,Initial_Fee_Remaining,Initial_Fee_On_Schedule,Royalty_Paid_Expected,Royalty_Paid_Actual,Marketing_Paid_Expected,Marketing_Paid_Actual,Min_Royalty_Biweekly,Missed_Marketing_Payment_Streak
0,STORE_0,2,1,1,29850,10000,Yes,19046,750,900,6000,750,2


In [5]:
pay["Missed_Payments_YTD"]=pay["Missed_Payments_6mo"]+pay["Missed_Payments_3mo"]
pay.head(1)

,Store_ID,Missed_Payments_12mo,Missed_Payments_6mo,Missed_Payments_3mo,Initial_Fee_Paid,Initial_Fee_Remaining,Initial_Fee_On_Schedule,Royalty_Paid_Expected,Royalty_Paid_Actual,Marketing_Paid_Expected,Marketing_Paid_Actual,Min_Royalty_Biweekly,Missed_Marketing_Payment_Streak,Missed_Payments_YTD
0,STORE_0,2,1,1,29850,10000,Yes,19046,750,900,6000,750,2,2


In [6]:
missed=pay[["Store_ID","Missed_Payments_12mo","Missed_Payments_YTD","Missed_Payments_6mo","Missed_Payments_3mo"]]
missed.head(1)

,Store_ID,Missed_Payments_12mo,Missed_Payments_YTD,Missed_Payments_6mo,Missed_Payments_3mo
0,STORE_0,2,2,1,1


In [7]:
mo12=missed.sort_values(by="Missed_Payments_12mo", ascending=False)
ytd=missed.sort_values(by="Missed_Payments_YTD",ascending=False)
mo6=missed.sort_values(by="Missed_Payments_6mo",ascending=False)
mo3=missed.sort_values(by="Missed_Payments_3mo",ascending=False)

ytd.head(1)

,Store_ID,Missed_Payments_12mo,Missed_Payments_YTD,Missed_Payments_6mo,Missed_Payments_3mo
155,STORE_155,0,6,2,4


In [8]:
initialf=pay[["Store_ID","Initial_Fee_Paid","Initial_Fee_Remaining"]]
initialf.head(1)

,Store_ID,Initial_Fee_Paid,Initial_Fee_Remaining
0,STORE_0,29850,10000


In [9]:
royalty=pay[["Store_ID","Royalty_Paid_Expected","Royalty_Paid_Actual"]]
royalty["dif"]=royalty["Royalty_Paid_Expected"]-royalty["Royalty_Paid_Actual"]
royalty.head(1)

C:\Users\silve\AppData\Local\Temp\ipykernel_7296\1612852311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  royalty["dif"]=royalty["Royalty_Paid_Expected"]-royalty["Royalty_Paid_Actual"]


,Store_ID,Royalty_Paid_Expected,Royalty_Paid_Actual,dif
0,STORE_0,19046,750,18296


In [10]:
royalty=royalty.sort_values(by="dif", ascending=False)
royalty=royalty.head(10)
royalty.head(1)

,Store_ID,Royalty_Paid_Expected,Royalty_Paid_Actual,dif
250,STORE_250,19909,1000,18909


In [ ]:
# Dashboard for visualization 

In [11]:
import dash
from jupyter_dash import JupyterDash
from dash import dcc,html
from dash.dependencies import Input,Output
import plotly.express as px 

In [16]:
app=dash.Dash(__name__)
app.title="Payments"

In [17]:
app.layout=html.Div([
    html.H1("Payments"),

    html.Label("Choose a column for Visualization"),

    dcc.Dropdown(
        id="column-dropdown",
        options=[
                 {"label":"Histogram of missed payments","value":1},
                 {"label":"Potential Audit Stores","value":2}
                ],
        value=1
    ),
    dcc.Graph(id="eda-graph")
])

In [18]:
@app.callback(
    Output("eda-graph","figure"),
    Input("column-dropdown","value")
)

def update_figure(selected_column):
    if selected_column==1:
        fig=px.histogram(ytd,x="Missed_Payments_YTD")
    elif selected_column==2:
        fig=px.bar(royalty, x="Store_ID",y="dif")
    else:
        fig=px.scatter(royalty, x="Store_ID",y="dif")
    return fig


app.run_server(port=8054,mode="external")